# Test on save and load of the custom keras model

We create a model, train it and save it in a given directory. Then we load it again, check if the score are the same on the dataset, and check that we can train it again.

In [1]:
# Putida
# dataset_file = "/Dataset/IJN1463_10_UB.npz"
# objective=['BIOMASS_KT2440_WT3']
# epoch = 20
# batch_size = 30
# uptake_max_index = None
# model_file = "/Models/AMNWt_IJN1463_10_UB.keras"

# E-coli
dataset_file = "/Dataset/iML1515_UB.npz"
objective=['BIOMASS_Ec_iML1515_core_75p37M']
epoch = 10
batch_size = 30
uptake_max_index = None
model_file = "/AMNWt_iML1515_UB.keras"

In [2]:
import tensorflow as tf
from amn.model.aMNWtModel import AMNWtModel, RNNCell

seed = 10
tf.random.set_seed(seed)

data_dir = "../data"
model_dir = "../models"
model_class = AMNWtModel

# Create the model with dataset
model = model_class(dataset_file=data_dir + dataset_file, 
                   objective=objective,
                   timestep=4,
                   hidden_dim=50,
                   verbose=True)
model.printout()

# Preprocessing
model.train_test_split(test_size=0.1, random_state=seed)
model.preprocessing_for_specific_model()

number of metabolites:  1877
filtered measurements size:  1
dataset file: ../data/Dataset/iML1515_UB.npz
model type: AMNWt
model medium bound: UB
timestep: 4
training set size (11000, 38) (11000, 1)
nbr hidden layer: 1
hidden layer size: 50
activation function: relu


In [3]:
# Build and train the model
AMNWt_model = model.build_model()
history = AMNWt_model.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model.predict(model.X_test)))

2023-11-29 15:32:04.303615: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 15:32:04.305107: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-11-29 15:32:04.902238: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


R2 : 0.9993539978754362
Q2 : 0.9996111356698927


In [4]:
# Save the train AMNWt_model
tf.keras.models.save_model(AMNWt_model,
                           model_dir +model_file, 
                           overwrite=True, 
                           save_format=None, 
                           save_traces=True)

In [5]:
from amn.tools import custom_loss

# Load the model
AMNWt_model_ = tf.keras.models.load_model(model_dir +model_file, 
                                          custom_objects={"RNNCell":RNNCell,
                                                          "my_mse":custom_loss(model.S, 
                                                                               model.P_out,
                                                                               model.P_in)}
                                          )

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9993539978754362
Q2 : 0.9996111356698927


In [6]:
# Train mode the model
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)
print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9991373357262987
Q2 : 0.9994145691035303


Annex : create a new model using the config file.

In [7]:
from amn.tools import custom_loss

seed = 10
tf.random.set_seed(seed)

# Recreate new model from config file First test on config.
config = AMNWt_model.get_config()
AMNWt_model_= tf.keras.Model.from_config(config, custom_objects={"RNNCell":RNNCell})
my_mse = custom_loss(model.S, model.P_out,model.P_in)

# Compile and train
AMNWt_model_.compile(loss=my_mse,optimizer='adam',metrics=[my_mse])
history = AMNWt_model_.fit(model.X_train, model.Y_train, epochs=epoch, batch_size=batch_size, verbose=0)

print("R2 :", model.R2(model.Y_train, AMNWt_model_.predict(model.X_train)))
print("Q2 :", model.R2(model.Y_test, AMNWt_model_.predict(model.X_test)))

R2 : 0.9993539978754362
Q2 : 0.9996111356698927
